In [4]:
import pandas as pd 
from gensim import corpora, models, similarities

In [5]:
review = pd.read_pickle('../output/bar_reviews_cleaned_and_tokenized.pickle')

## Now we must generate a dictionary which maps vocabulary into a number

In [ ]:
import time 
from itertools import chain



print 'Generating vector dictionary....'
review_flatten = list(chain.from_iterable(review.cleaned_tokenized.iloc[:]))
start = time.time()
id2word_wiki = corpora.Dictionary(review_flatten)
print 'Dictonary generated in %1.2f seconds'%(time.time()-start)


Generating vector dictionary....
Dictonary generated in 45.46 seconds


In [ ]:

# Convert corpus to bag of words for use with gensim...
# See https://radimrehurek.com/gensim/tut1.html#from-strings-to-vectors
corpus = map(lambda doc: id2word_wiki.doc2bow(doc), review_flatten)
corpora.MmCorpus.serialize('../output/bar_corpus.mm', corpus)


# Can laod the corpus with 
# from gensim import corpora
# corpus = corpora.MmCorpus('../output/bar_corpus.mm')

